скачиваем библиотеки

In [42]:
!pip install psycopg2
!pip install pandas
!pip install scipy

  Using cached scipy-1.12.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached scipy-1.12.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (38.4 MB)


испортируем библиотеки

In [2]:
import pandas as pd

# 1.1 Загрузка данных

загружаем даннные

In [56]:
exoenses = pd.read_csv('expenses.csv')
orders = pd.read_csv('orders.csv')
visitation = pd.read_csv('visitation.csv')

смотрим

In [4]:
exoenses

,dt,Channel,costs
0,2022-05-01,FaceBoom,113.3
1,2022-05-02,FaceBoom,78.1
2,2022-05-03,FaceBoom,85.8
3,2022-05-04,FaceBoom,136.4
4,2022-05-05,FaceBoom,122.1
...,...,...,...
1795,2022-10-23,lambdaMediaAds,4.0
1796,2022-10-24,lambdaMediaAds,6.4
1797,2022-10-25,lambdaMediaAds,8.8
1798,2022-10-26,lambdaMediaAds,8.8


In [30]:
exoenses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   dt       1800 non-null   object 
 1   Channel  1800 non-null   object 
 2   costs    1800 non-null   float64
dtypes: float64(1), object(2)
memory usage: 42.3+ KB


In [5]:
orders

,User Id,Event Dt,Revenue
0,188246423999,2022-05-01 23:09:52,4.99
1,174361394180,2022-05-01 12:24:04,4.99
2,529610067795,2022-05-01 11:34:04,4.99
3,319939546352,2022-05-01 15:34:40,4.99
4,366000285810,2022-05-01 13:59:51,4.99
...,...,...,...
40207,651604369137,2022-10-31 16:19:07,4.99
40208,275341387049,2022-10-31 01:17:17,4.99
40209,374656616484,2022-10-31 06:17:29,4.99
40210,168548862926,2022-10-31 22:46:19,4.99


In [31]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40212 entries, 0 to 40211
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   User Id   40212 non-null  int64  
 1   Event Dt  40212 non-null  object 
 2   Revenue   40212 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 942.6+ KB


In [6]:
visitation

,User Id,Region,Device,Channel,Session Start,Session End
0,981449118918,Russia,iPhone,organic,2022-05-01 02:36:01,2022-05-01 02:45:01
1,278965908054,Russia,iPhone,organic,2022-05-01 04:46:31,2022-05-01 04:47:35
2,590706206550,Russia,Mac,organic,2022-05-01 14:09:25,2022-05-01 15:32:08
3,326433527971,Russia,Android,TipTop,2022-05-01 00:29:59,2022-05-01 00:54:25
4,349773784594,Russia,Mac,organic,2022-05-01 03:33:35,2022-05-01 03:57:40
...,...,...,...,...,...,...
309896,329994900775,Turkey,PC,LeapBob,2022-10-31 13:28:12,2022-10-31 14:39:29
309897,334903592310,Thailand,PC,lambdaMediaAds,2022-10-31 22:14:52,2022-10-31 22:39:36
309898,540102010126,China,PC,organic,2022-10-31 01:40:48,2022-10-31 01:41:31
309899,308736936846,China,Mac,organic,2022-10-31 07:37:34,2022-10-31 07:37:55


присоединяемся к database

который я создал в pgAdmin 4

In [87]:
import psycopg2;

connection = psycopg2.connect(database="Users",host="localhost",user="postgres",password="1234")

создаем таблицы в pgAdmin 4 для данных

In [83]:
cursor = connection.cursor()
cursor.execute('CREATE TABLE IF NOT EXISTS public.expenses(dt text COLLATE pg_catalog."default",channel character varying(30) COLLATE pg_catalog."default",costs character varying(30) COLLATE pg_catalog."default")    TABLESPACE pg_default;  ALTER TABLE IF EXISTS public.expenses    OWNER to postgres;')
connection.close()

In [86]:
cursor = connection.cursor()
cursor.execute('CREATE TABLE IF NOT EXISTS public.orders("User Id" text COLLATE pg_catalog."default","Event Dt" text COLLATE pg_catalog."default","Revenue" text COLLATE pg_catalog."default")    TABLESPACE pg_default;  ALTER TABLE IF EXISTS public.orders    OWNER to postgres;')
connection.close()

In [88]:
cursor = connection.cursor()
cursor.execute('CREATE TABLE IF NOT EXISTS public.visitation("User Id" text COLLATE pg_catalog."default","Region" text COLLATE pg_catalog."default","Device" text COLLATE pg_catalog."default","Channel" text COLLATE pg_catalog."default","Session Start" text COLLATE pg_catalog."default","Session End" text COLLATE pg_catalog."default")    TABLESPACE pg_default;  ALTER TABLE IF EXISTS public.visitation    OWNER to postgres;')
connection.close()

данные спокойно загрузились

# 1.2 Предварительная обработка данных

Делаем предварительную обработку данных для каждого dataframe

## expenses

смотрим на данные

In [57]:
exoenses

,dt,Channel,costs
0,2022-05-01,FaceBoom,113.3
1,2022-05-02,FaceBoom,78.1
2,2022-05-03,FaceBoom,85.8
3,2022-05-04,FaceBoom,136.4
4,2022-05-05,FaceBoom,122.1
...,...,...,...
1795,2022-10-23,lambdaMediaAds,4.0
1796,2022-10-24,lambdaMediaAds,6.4
1797,2022-10-25,lambdaMediaAds,8.8
1798,2022-10-26,lambdaMediaAds,8.8


удаляем дубликаты если они были

In [58]:
exoenses.drop_duplicates()

,dt,Channel,costs
0,2022-05-01,FaceBoom,113.3
1,2022-05-02,FaceBoom,78.1
2,2022-05-03,FaceBoom,85.8
3,2022-05-04,FaceBoom,136.4
4,2022-05-05,FaceBoom,122.1
...,...,...,...
1795,2022-10-23,lambdaMediaAds,4.0
1796,2022-10-24,lambdaMediaAds,6.4
1797,2022-10-25,lambdaMediaAds,8.8
1798,2022-10-26,lambdaMediaAds,8.8


смотрим на среднее значение для просмотра выбросов

In [51]:
exoenses.describe()

,costs
count,1800.000000
mean,58.609611
std,107.740223
min,0.800000
25%,6.495000
50%,12.285000
75%,33.600000
max,630.000000


удаяем выбросы

In [64]:
exoenses = exoenses.loc[exoenses['costs'] <= 60]
 
print(exoenses)

              dt         Channel  costs
180   2022-05-01    MediaTornado   6.24
181   2022-05-02    MediaTornado   5.04
182   2022-05-03    MediaTornado   6.96
183   2022-05-04    MediaTornado   9.36
184   2022-05-05    MediaTornado  11.04
...          ...             ...    ...
1795  2022-10-23  lambdaMediaAds   4.00
1796  2022-10-24  lambdaMediaAds   6.40
1797  2022-10-25  lambdaMediaAds   8.80
1798  2022-10-26  lambdaMediaAds   8.80
1799  2022-10-27  lambdaMediaAds  12.00

[1448 rows x 3 columns]


смотрим опять

In [65]:
exoenses.describe()

,costs
count,1448.000000
mean,12.942265
std,10.218084
min,0.800000
25%,5.757500
50%,9.750000
75%,16.380000
max,60.000000


## orders

смотрим на данные

In [66]:
orders

,User Id,Event Dt,Revenue
0,188246423999,2022-05-01 23:09:52,4.99
1,174361394180,2022-05-01 12:24:04,4.99
2,529610067795,2022-05-01 11:34:04,4.99
3,319939546352,2022-05-01 15:34:40,4.99
4,366000285810,2022-05-01 13:59:51,4.99
...,...,...,...
40207,651604369137,2022-10-31 16:19:07,4.99
40208,275341387049,2022-10-31 01:17:17,4.99
40209,374656616484,2022-10-31 06:17:29,4.99
40210,168548862926,2022-10-31 22:46:19,4.99


смотрим на пропуски

In [67]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40212 entries, 0 to 40211
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   User Id   40212 non-null  int64  
 1   Event Dt  40212 non-null  object 
 2   Revenue   40212 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 942.6+ KB


удаляем дубликаты

In [89]:
orders.drop_duplicates()

,User Id,Event Dt,Revenue
0,188246423999,2022-05-01 23:09:52,4.99
1,174361394180,2022-05-01 12:24:04,4.99
2,529610067795,2022-05-01 11:34:04,4.99
3,319939546352,2022-05-01 15:34:40,4.99
4,366000285810,2022-05-01 13:59:51,4.99
...,...,...,...
40207,651604369137,2022-10-31 16:19:07,4.99
40208,275341387049,2022-10-31 01:17:17,4.99
40209,374656616484,2022-10-31 06:17:29,4.99
40210,168548862926,2022-10-31 22:46:19,4.99


смотрим на среднее значение для просмотра выбросов

In [68]:
orders.describe()

,User Id,Revenue
count,4.021200e+04,40212.000000
mean,4.990295e+11,5.370608
std,2.860937e+11,3.454208
min,5.993260e+05,4.990000
25%,2.511324e+11,4.990000
50%,4.982840e+11,4.990000
75%,7.433327e+11,4.990000
max,9.998954e+11,49.990000


Удаляем выбросы

In [72]:
orders = orders.loc[orders['Revenue'] <= 15]
 
print(orders)

            User Id             Event Dt  Revenue
0      188246423999  2022-05-01 23:09:52     4.99
1      174361394180  2022-05-01 12:24:04     4.99
2      529610067795  2022-05-01 11:34:04     4.99
3      319939546352  2022-05-01 15:34:40     4.99
4      366000285810  2022-05-01 13:59:51     4.99
...             ...                  ...      ...
40207  651604369137  2022-10-31 16:19:07     4.99
40208  275341387049  2022-10-31 01:17:17     4.99
40209  374656616484  2022-10-31 06:17:29     4.99
40210  168548862926  2022-10-31 22:46:19     4.99
40211  329994900775  2022-10-31 13:29:06     4.99

[39796 rows x 3 columns]


смотрим опять

In [73]:
orders.describe()

,User Id,Revenue
count,3.979600e+04,39796.000000
mean,4.989206e+11,5.057972
std,2.861736e+11,0.506799
min,5.993260e+05,4.990000
25%,2.507097e+11,4.990000
50%,4.980662e+11,4.990000
75%,7.431993e+11,4.990000
max,9.998954e+11,9.990000


## visitation

смотрим на данные

In [74]:
visitation

,User Id,Region,Device,Channel,Session Start,Session End
0,981449118918,Russia,iPhone,organic,2022-05-01 02:36:01,2022-05-01 02:45:01
1,278965908054,Russia,iPhone,organic,2022-05-01 04:46:31,2022-05-01 04:47:35
2,590706206550,Russia,Mac,organic,2022-05-01 14:09:25,2022-05-01 15:32:08
3,326433527971,Russia,Android,TipTop,2022-05-01 00:29:59,2022-05-01 00:54:25
4,349773784594,Russia,Mac,organic,2022-05-01 03:33:35,2022-05-01 03:57:40
...,...,...,...,...,...,...
309896,329994900775,Turkey,PC,LeapBob,2022-10-31 13:28:12,2022-10-31 14:39:29
309897,334903592310,Thailand,PC,lambdaMediaAds,2022-10-31 22:14:52,2022-10-31 22:39:36
309898,540102010126,China,PC,organic,2022-10-31 01:40:48,2022-10-31 01:41:31
309899,308736936846,China,Mac,organic,2022-10-31 07:37:34,2022-10-31 07:37:55


смотрим на пустые значения

In [75]:
visitation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309901 entries, 0 to 309900
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   User Id        309901 non-null  int64 
 1   Region         309901 non-null  object
 2   Device         309901 non-null  object
 3   Channel        309901 non-null  object
 4   Session Start  309901 non-null  object
 5   Session End    309901 non-null  object
dtypes: int64(1), object(5)
memory usage: 14.2+ MB


смотрим на среднее значение 

In [76]:
visitation.describe()

,User Id
count,3.099010e+05
mean,4.997664e+11
std,2.887899e+11
min,5.993260e+05
25%,2.493691e+11
50%,4.989906e+11
75%,7.495211e+11
max,9.999996e+11


выбросов нет

Сохраняем датафрейм 

In [77]:
exoenses.to_csv('expenses_new.csv',index=False)
orders.to_csv('orders_new.csv',index=False)
visitation.to_csv('visitation_new.csv',index=False)

Загружаем их в Databases

# 1.3 Задание функций расчета ключевых показателей

Сделаем функцию kall который будет формировать dataframe который включает поля: дата и время начала сессии при первом
посещении каждого пользователя, устройство, регион, рекламный источник,дате первого посещения и о
средней стоимости привлечения пользователей в день.

In [103]:
def kall():
    exoenses1 = exoenses.drop(columns=['Channel'])
    exoenses2 = exoenses1.drop(columns=['dt'])
    visitation1 = pd.concat([visitation, exoenses2], axis=1)
    vis = visitation1.fillna(0)
    vis

# 1.4 Подготовка отчета

В результате этой сессии почти все задачи были выполнены

1.1 Загрузка данных
- Была создана база данных Users, содержащая структуру, согласно исходным файлам.
Выполнено подключение к базе данных Users. Извлечено необходимые для
исследования данные из базы данных (visitation, orders, expenses).
Предусмотрена обработка исключений при загрузке данных.
Все данные представлены в виде DataFrame — объекта для
представления табличных данных в Pandas.

1.2 Предварительная обработка данных
- Выполнена предварительная обработка загруженных данных. Произведен анализ
и обработка пропусков, дубликатов, аномальных значений. Приведены данные к
приемлемому формату. Сохранены данные в формате csv и обновлены данные в
используемой базе данных для дальнейшего исследования.

1.3 Задание функций расчета ключевых показателей
- Задана функция для формирования набора данных с профилями пользователей.
Набор включает поля: дата и время начала сессии при первом
посещении каждого пользователя, устройство, регион, рекламный источник,дата первого посещения и о
средней стоимости привлечения пользователей в день.
Определена функция для расчёта прибыли от клиентов и коэффициента
окупаемости вложений (LTV, ROI), расчёта удержания и конверсии.

1.4 Подготовка отчета
- Был подготовлен отчет о проделанной работе по итогам сессии, в котором
представлены результаты, выводы и обоснования выбора по каждому разделу
задания. Результаты работы состоят из отчетов в форматах: .ipynb и
.html, а также исходников с возможностью перекомпиляции. Архив Data.zip
содержит все результаты выполнения модуля, а также все необходимые
файлы для запуска и проверки участков кода. В файле Readme.txt
описано содержимое результирующих файлов архива Data.zip.